## Post-procesamiento de la imagen de prediccion de modelo de inundaciones

En este notebook se presentan los pasos seguidos para el post-procesamiento de los resultados predichos del modelo de area suscpetibles a inundacion. Estos pasos incluyen:

1. Filtro por umbral de probabilidad;

2. Poligonizacion y filtro por area;

3. Extraccion de resultados en base a raster original; 

4. Extraccion de resultados por municipio.

## 1. Importacion de librerias y definicion de variables

In [ ]:
#!pip install geopandas

In [ ]:
from satproc.postprocess.polygonize import polygonize 
from satproc.filter import filter_by_max_prob
import subprocess
import os
import geopandas as gpd

In [ ]:
#bucket con el archivo raster del resultado de la prediccion sin post-procesamiento
BUCKET_PRED = "gs://dym-indunor-temp/immap/v3_1/floods_annotations/results/pred_500/*.tif"

#bcuket con archivo vectorial de municipios para recorte final del resultado de prediccion
BUCKET_MUN = "gs://dym-indunor-temp/immap/v3_1/data/munis_v3.gpkg"

#creamos la carpeta a donde descargaremos la prediccion 
BASE_PATH = "../"
PATH_PRED_FILES = f"{BASE_PATH}/flood_pred/"
PATH_MUN_FILES = f"{BASE_PATH}/MUN_files/"

## 1.3. Descarga de archivos desde bucket

In [ ]:
#RASTER PREDICCION
!mkdir -p $PATH_PRED_FILES   #crea la carpeta
!gsutil -m cp -r $BUCKET_PRED   $PATH_PRED_FILES # descarga de imagenes

#MUNICIPIOS
!mkdir -p $PATH_MUN_FILES   #crea la carpeta
!gsutil -m cp -r $BUCKET_MUN   $PATH_MUN_FILES # descarga de archivos

## 1.4. Filtro por umbral de probabilidad

In [ ]:
#definimos el umbral a la probabilidad
THR = 0.2
str_thr=str(int(THR*10))
thr1=str(int(THR*255))

input_pred = f'{PATH_PRED_FILES}all_v0.tif'
output_pred_threshold = f'{PATH_PRED_FILES}all_v0_threshold.tif'
exp = "((A >"+thr1+")*199)+1"
cmd_calc = f'gdal_calc.py -A {input_pred} --A_band=1 --outfile {output_pred_threshold} --calc="{exp}" --NoDataValue=0'
subprocess.run(cmd_calc, shell=True)
print(cmd_calc)
print("--",output_tif_path)
print("<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<") 

## 1.5. Poligonizado de las areas con pb mayor al umbral

In [ ]:
output_shp_path = f'{PATH_PRED_FILES}all_v0_threshold_poligonized.gpkg'
cmd_polygonize = f'gdal_polygonize.py {output_pred_threshold} {output_shp_path}'
subprocess.run(cmd_polygonize, shell=True)

## 1.6. Filtro por area minima

In [ ]:
#reproyeccion a UTM, A fin de poder estimar las areas de los poligonos y filtrar poligonos chicos (a fin de eliminar falsos positivos)
output_shp_path_pr = f'{output_shp_path[0:-5]}_epsg32618.gpkg'
!ogr2ogr -s_srs EPSG:4326 -t_srs EPSG:32618 -f 'GPKG' $output_shp_path_pr $output_shp_path
print('<<<<<<<<<<<<',dst_file)

In [ ]:
min_area=750000 

output_area_min =  f'{PATH_PRED_FILES}all_v0_threshold_poligonized_epsg32618_minarea.gpkg

!ogr2ogr \
    -t_srs EPSG:32618 \
    -f "GPKG" \
    -sql "SELECT * FROM out m WHERE (ST_Area(geom) > $min_area2)" \
    -dialect SQLITE \
    -nln results \
    $output_area_min \
    $output_shp_path_pr
print(">>>>>>>>",output_path)

## 1.7. Recorte del raster de prediccion original por los poligonos mayores al area minima

In [ ]:
final_output_pred = f'{PATH_PRED_FILES}all_v0_threshold_ab750000.tif'
cmd_clip_raster = f"gdalwarp -cutline {output_area_min} -crop_to_cutline -dstalpha {input_pred} {final_output_pred}"
subprocess.run(cmd_clip_raster, shell=True)
print(minarea_raster)

## 1.8. Recorte final por municipio 

In [ ]:
#import geopandas as gpd
for file in os.listdir(PATH_MUN_FILES):
    mun = gpd.read_file(Path(f'{PATH_MUN_FILES}{file}'))
    for i, row in mun.iterrows():
        municipio = row['COD_MUNICIPIO']
        dpto = row['DPTO_COD']
        outpath_folder = f'{PATH_MUN_FILES}{dpto}/{municipio}/'
        os.makedirs(outpath_folder, exist_ok=True)
        outpath_municipio = f'{outpath_folder}{municipio}.geojson'
        gpd.GeoDataFrame(geometry=list(row.geometry)).to_file(outpath_municipio)
        
        pred_raster_folder = f'{PATH_PRED_FILES}FINAL/{dpto}/{municipio}/'
        os.makedirs(mask_folder, exist_ok=True)

        for file in os.listdir(outpath_folder):
            search_criteria_geojson = f'{outpath_folder}*.geojson'
            subset_files = glob.glob(search_criteria_geojson)
            
            for fl in subset_files:
                outfile = f'{pred_raster_folder}{municipio}.tif'
                ds = gdal.Open(final_output_pred)
                ds_recorte = gdal.Warp(outfile, ds, cutlineDSName = fl, cropToCutline = True, dstNodata = np.nan)
                ds_recorte = None